# Careplan generation: Generating labels

**Author:** Eva Rombouts  
**Date:** 2024-07-21  
**Version:** 0.1

### Description


### Imports and constants

In [2]:
# Environment setup
import os

def check_environment():
    try:
        import google.colab
        return "Google Colab"
    except ImportError:
        return "Local Environment"

env = check_environment()

if env == "Google Colab":
    print("Running in Google Colab")
    !pip install -q langchain langchain_core langchain_openai langchain_community
    from google.colab import drive, userdata
    drive.mount('/content/drive')
    os.chdir('/content/drive/My Drive/Colab Notebooks/GenCareAI/scripts')
    OPENAI_API_KEY = userdata.get('GCI_OPENAI_API_KEY')
else:
    print("Running in Local Environment")
    # !pip install -q
    from dotenv import load_dotenv
    load_dotenv()
    OPENAI_API_KEY = os.getenv('GCI_OPENAI_API_KEY')


Running in Local Environment


In [3]:
import os
import pandas as pd
import json
from tqdm import tqdm  
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_community.callbacks import get_openai_callback

In [4]:
# Constants and Configurations
MODEL = 'gpt-4o-mini-2024-07-18'
TEMP = 1.1

### Data

In [41]:
df = pd.read_csv('../data/df_careplans.csv')

In [6]:
class CarePlanItem(BaseModel):
    probleem: str
    doel: str
    acties: List[str]

class CarePlan(BaseModel):
    zorgplan: List[CarePlanItem]

In [7]:
model= ChatOpenAI(api_key=OPENAI_API_KEY, temperature=TEMP, model=MODEL)

In [8]:
pyd_parser = PydanticOutputParser(pydantic_object=CarePlan)

In [26]:
test = df['notes'][0]

In [27]:
PT = PromptTemplate(
    template = """
Hieronder staan rapportages van een verpleeghuisclient met dementie. 
Schrijf een zorgplan voor de drie belangrijkste zorgproblemen op basis van de rapportages.
De acties moeten zijn beschreven in de rapportages.

RAPPORTAGES:
{notes}

{format_instructions}
""",
    input_variables=["notes"],
    partial_variables={"format_instructions": pyd_parser.get_format_instructions()},
)

# Format the prompt for the example library
P = PT.format(notes=test)
print(P)


Hieronder staan rapportages van een verpleeghuisclient met dementie. 
Schrijf een zorgplan voor de drie belangrijkste zorgproblemen op basis van de rapportages.
De acties moeten zijn beschreven in de rapportages.

RAPPORTAGES:
- Meneer heeft vanochtend hulp gehad bij het aankleden en is daarna gaan ontbijten. Hij had wat moeite met het herkennen van zijn kleding, maar kon uiteindelijk zelfstandig eten.
- Tijdens de lunch had meneer wat hulp nodig bij het snijden van zijn eten. Hij was rustig en genoot van zijn maaltijd. Diabetesmanagement werd in de gaten gehouden.
- In de middag heeft meneer deelgenomen aan een puzzelactiviteit en was hier erg geconcentreerd mee bezig. Zijn mobiliteit met de rollator werd regelmatig gecheckt.
- Vanmorgen had meneer wat hulp nodig bij de toiletgang en het wassen. Zijn bloeddruk en bloedsuiker werden gecontroleerd en waren binnen normale waarden.
- Tijdens de lunch was meneer wat vergeetachtig en wist niet meer wat voor dag het was. Hij had ondersteuni

In [28]:
chain = PT | model | pyd_parser

In [29]:
result = chain.invoke({"notes": test})

In [33]:
print(result)

zorgplan=[CarePlanItem(probleem='Agitatie en agressie', doel='Meneer Leo kan zijn emoties beter reguleren en vertoont minder agressief gedrag', acties=['Extra toezicht inschakelen om escalatie te voorkomen.', 'Overleggen met arts over medicatie om agressie te verminderen.', 'Meneer naar zijn kamer begeleiden en rust bieden door bekende muziek af te spelen.', 'Kalmerende gesprekken voeren en rustmomenten inlassen bij onrust.']), CarePlanItem(probleem='Moeite met eten en slikken', doel='Meneer Leo kan zelfstandig en zonder begeleiding eten en drinken.', acties=['Extra hulp aanbieden bij het snijden van zijn eten.', 'Zachte voeding en kleinere hapjes aanbieden om slikproblemen te beperken.', 'Afleiding bieden met muziek om te zorgen dat hij rustig kan eten.']), CarePlanItem(probleem='Angst en hallucinaties', doel='Meneer Leo ervaart minder angst en hallucinaties worden verminderd.', acties=['Overleggen met een psycholoog om geschikte interventies te vinden.', 'Kalmerende muziek afspelen e

In [30]:
# Print het resultaat in JSON-formaat
print(result.json(indent=2))

{
  "zorgplan": [
    {
      "probleem": "Moeite met Aankleden en Persoonlijke Verzorging",
      "doel": "Meneer kan zelfstandig aankleden met minimale ondersteuning.",
      "acties": [
        "Bied extra tijd en geduld tijdens het aankleden.",
        "Geef ondersteuning bij het kiezen van kleding als meneer moeite heeft met herkennen.",
        "Help meneer met wassen en scheren indien nodig."
      ]
    },
    {
      "probleem": "Vergeetachtigheid en Verward Gedrag",
      "doel": "Meneer kan zijn dagelijkse activiteiten zelfstandig uitvoeren met minimale begeleiding.",
      "acties": [
        "Monitor en ondersteun meneer bij maaltijden om verward gedrag te verminderen.",
        "Bied geruststelling en afleiding met herkenbare activiteiten zoals puzzelen of geheugentraining.",
        "Controleer en bespreek regelmatig de datum en tijd om mnre te helpen herinneren."
      ]
    },
    {
      "probleem": "Diabetesmanagement",
      "doel": "Bloedglucosewaarden blijven stab

In [42]:
df = df.sample(2)

In [43]:
with get_openai_callback() as cb:
    data = []
    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):  # Gebruik tqdm om de progress bar te tonen
        result = chain.invoke({"notes": row['notes']})
        for item in result.zorgplan:
            data.append({
                'ct_id': row['ct_id'],
                'month': row['month'],
                'probleem': item.probleem,
                'doel': item.doel,
                'acties': '; '.join(item.acties)  # Acties als een samengevoegde string
            })

    # Opslaan van de verzamelde data in JSON formaat
    with open('../data/df_careplans_generated.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print("Data is succesvol opgeslagen in JSON formaat.")
    print(cb)

100%|██████████| 2/2 [00:10<00:00,  5.16s/it]

Data is succesvol opgeslagen in JSON formaat.
Tokens Used: 4357
	Prompt Tokens: 3730
	Completion Tokens: 627
Successful Requests: 2
Total Cost (USD): $0.0
